在沙盒环境，使用期货原始因子进行线性信号转化

In [1]:
import os,pdb,itertools,copy,datetime,sys,collections
os.environ['ULTRON_DATA'] = 'keim'

In [2]:
import numpy as np
import pandas as pd

In [3]:
from ultron.env import *
from ultron.strategy.builder.factor import ts_zscore_normalization,cross_section_scale
from ultron.strategy.builder.signal import calc_weighted_signal,calc_1k_signal,calc_composite_signal,\
    calc_factors_score_signal

/var/log/ultron/2022-09-27.log


In [4]:
enable_example_env()

2022-09-27 18:27:35,600 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/keim


#### 读取因子

In [5]:
total_data = pd.read_csv(os.path.join(g_project_data, 'factor.csv'), index_col=0)
total_data['trade_date'] = pd.to_datetime(total_data['trade_date'])
factors_data = total_data[['trade_date','code','WeightShortVolRelTotIntChg','BM_RecentSecond_20D']]
factors_data.head()

,trade_date,code,WeightShortVolRelTotIntChg,BM_RecentSecond_20D
0,2017-10-27,A,-0.037579,-0.023047
1,2017-10-27,AL,-0.012891,0.001133
2,2017-10-27,BU,0.275875,0.069999
3,2017-10-27,C,0.245555,-0.001541
4,2017-10-27,CF,-0.131799,0.004838


#### 时序标准化

In [6]:
def normalization(factors_data, name, bound=3, winsize=120):
    factor_data = factors_data.set_index(['trade_date','code'])[name].dropna().unstack()
    rval = ts_zscore_normalization(data=factor_data, bound=bound, winsize=winsize)
    return rval

In [7]:
factor_data1 = normalization(factors_data=factors_data, name='WeightShortVolRelTotIntChg')
factor_data2 = normalization(factors_data=factors_data, name='BM_RecentSecond_20D')

#### 横截面标准化

In [8]:
factor_data1 = cross_section_scale(factor_data1)
factor_data2 = cross_section_scale(factor_data2)

### 多因子信号合成

#### 构建数据集

In [9]:
signals = collections.OrderedDict()

In [10]:
signals['WeightShortVolRelTotIntChg'] = factor_data1
signals['BM_RecentSecond_20D'] = factor_data2
factor_list = ['WeightShortVolRelTotIntChg','BM_RecentSecond_20D']
factor_weight = [0.5,0.5]

#### 因子加权

In [11]:
weigthed = calc_weighted_signal(signals, factor_list, factor_weight)
weigthed.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.025657,0.032708,0.011506,0.012441,0.020483,-0.001438,-0.003308,...,0.018207,-0.020650,0.012860,-0.012399,0.038972,-0.038019,-0.026032,-0.007904,-0.008203,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.047116,0.019029,-0.008148,0.021857,0.004359,-0.034689,0.015959,...,0.041486,-0.022772,0.018861,0.002266,0.024402,-0.020301,-0.000903,-0.015956,-0.006656,0.019837
2022-06-20,0.028062,0.035740,0.014332,-0.026098,0.030309,-0.001631,0.035979,0.020550,-0.031629,0.023821,...,0.047478,-0.054444,0.025348,-0.007312,0.022329,-0.013262,-0.002426,-0.012271,0.003843,0.066617
2022-06-21,0.028113,0.037144,0.024982,-0.013389,0.035868,0.002471,0.034907,-0.004691,-0.010700,0.006578,...,0.040327,-0.015019,0.030972,-0.001642,0.082832,-0.030429,-0.014258,-0.015524,-0.004235,0.064214
2022-06-22,0.029034,0.025558,0.023268,0.043909,0.036202,-0.006711,0.033628,-0.013291,-0.032340,0.000139,...,0.047920,-0.018767,0.018534,0.013492,0.084965,-0.031115,-0.005580,-0.013369,0.001593,0.067409


#### 1/K 

In [12]:
k1_data = calc_1k_signal(signals, factor_list, factor_weight, False)
k1_data.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.000000,0.062500,0.062500,0.000000,0.062500,0.0,0.0,0.062500,0.00,0.0,...,0.062500,0.040000,0.062500,0.0400,0.062500,0.040000,0.000000,0.000000,0.020000,0.000000
2022-06-17,0.000000,0.062500,0.062500,0.052632,0.062500,0.0,0.0,0.000000,0.00,0.0,...,0.062500,0.052632,0.062500,0.0625,0.000000,0.000000,0.000000,0.052632,0.026316,0.000000
2022-06-20,0.037037,0.037037,0.037037,0.062500,0.037037,0.0,0.0,0.037037,0.00,0.0,...,0.037037,0.062500,0.037037,0.0000,0.000000,0.000000,0.000000,0.000000,0.018519,0.037037
2022-06-21,0.038462,0.038462,0.038462,0.052632,0.038462,0.0,0.0,0.000000,0.00,0.0,...,0.038462,0.052632,0.038462,0.0000,0.038462,0.052632,0.052632,0.052632,0.026316,0.038462
2022-06-22,0.032258,0.032258,0.032258,0.032258,0.032258,0.0,0.0,0.050000,0.05,0.0,...,0.032258,0.000000,0.000000,0.0000,0.032258,0.050000,0.050000,0.050000,0.016129,0.032258


#### 复合信号

In [13]:
composite_data = calc_composite_signal(signals, factor_list, factor_weight, False)
composite_data.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.000000,0.048780,0.048780,0.000000,0.048780,0.0,0.0,0.048780,0.000000,0.0,...,0.048780,-0.048780,0.048780,-0.048780,0.048780,-0.048780,0.000000,0.000000,-0.024390,0.000000
2022-06-17,0.000000,0.057143,0.057143,-0.057143,0.057143,0.0,0.0,0.000000,0.000000,0.0,...,0.057143,-0.057143,0.057143,0.057143,0.000000,0.000000,0.000000,-0.057143,-0.028571,0.000000
2022-06-20,0.046512,0.046512,0.046512,-0.046512,0.046512,0.0,0.0,0.046512,0.000000,0.0,...,0.046512,-0.046512,0.046512,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.046512
2022-06-21,0.044444,0.044444,0.044444,-0.044444,0.044444,0.0,0.0,0.000000,0.000000,0.0,...,0.044444,-0.044444,0.044444,0.000000,0.044444,-0.044444,-0.044444,-0.044444,-0.022222,0.044444
2022-06-22,0.039216,0.039216,0.039216,0.039216,0.039216,0.0,0.0,-0.039216,-0.039216,0.0,...,0.039216,0.000000,0.000000,0.000000,0.039216,-0.039216,-0.039216,-0.039216,0.019608,0.039216


#### 评分信号

In [14]:
score_data = calc_factors_score_signal(signals, factor_list, factor_weight)
score_data.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.035714,0.035714,0.035714,-0.035714,0.035714,0.035714,0.035714,0.035714,0.000000,0.035714,...,0.035714,-0.035714,0.035714,-0.035714,0.035714,-0.035714,-0.035714,0.000000,0.000000,0.035714
2022-06-17,0.035714,0.035714,0.035714,-0.035714,0.035714,-0.035714,0.035714,0.000000,-0.035714,0.035714,...,0.035714,-0.035714,0.035714,0.000000,0.035714,-0.035714,0.000000,-0.035714,-0.035714,0.035714
2022-06-20,0.034483,0.034483,0.000000,-0.034483,0.034483,-0.034483,0.034483,0.034483,-0.034483,0.034483,...,0.034483,-0.034483,0.034483,-0.034483,0.034483,-0.034483,0.000000,-0.034483,0.000000,0.034483
2022-06-21,0.035714,0.035714,0.035714,-0.035714,0.035714,0.000000,0.035714,-0.035714,-0.035714,0.000000,...,0.035714,-0.035714,0.035714,0.000000,0.035714,-0.035714,-0.035714,-0.035714,0.000000,0.035714
2022-06-22,0.035714,0.035714,0.035714,0.035714,0.035714,-0.035714,0.035714,-0.035714,-0.035714,-0.035714,...,0.035714,-0.035714,0.000000,0.000000,0.035714,-0.035714,-0.035714,-0.035714,0.000000,0.035714


#### 平滑处理

In [15]:
rval = score_data.rolling(window=20, center=False, axis=0).mean()
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,-0.022549,0.000111,0.031354,0.020702,-0.003736,0.017257,0.000234,-0.001950,-0.015648,-0.011958,...,0.010242,0.007184,-0.003379,-0.006789,-0.007123,0.020825,-0.006893,0.020993,0.006666,0.000288
2022-06-17,-0.018977,0.003683,0.031354,0.018916,-0.000165,0.017257,0.000234,-0.000165,-0.017434,-0.008386,...,0.013813,0.003613,-0.003379,-0.008575,-0.005337,0.017253,-0.008679,0.017422,0.006666,0.000288
2022-06-20,-0.015529,0.007131,0.029630,0.015468,0.003284,0.015533,0.000234,0.003284,-0.019158,-0.004938,...,0.017262,0.000165,0.000069,-0.012023,-0.003613,0.013805,-0.010403,0.013974,0.006666,0.000288
2022-06-21,-0.012019,0.008917,0.029692,0.013682,0.005069,0.015533,0.000296,0.003222,-0.020944,-0.003214,...,0.020771,-0.003345,0.003579,-0.013748,-0.000103,0.010295,-0.013913,0.010464,0.004942,0.000349
2022-06-22,-0.010234,0.010702,0.029692,0.013682,0.006855,0.013748,0.000296,0.001436,-0.020944,-0.005000,...,0.024343,-0.006917,0.005365,-0.013748,0.003468,0.006724,-0.017484,0.006892,0.006728,0.000349
